<a href="https://colab.research.google.com/github/joooser/Natural_Language/blob/main/Modelos_clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de palabras (por género de nombre)

In [ ]:
import nltk, random
nltk.download('names')
from nltk.corpus import names 

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


**Función básica de extracción de atributos**

In [ ]:
# definición de atributos relevantes
def atributos(palabra):
	return {'ultima_letra': palabra[-1]}

tagset = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])

In [ ]:
tagset[:10]

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abbey', 'male'),
 ('Abbie', 'male'),
 ('Abbot', 'male'),
 ('Abbott', 'male'),
 ('Abby', 'male'),
 ('Abdel', 'male'),
 ('Abdul', 'male'),
 ('Abdulkarim', 'male')]

In [ ]:
random.shuffle(tagset)
tagset[:10]

[('Armond', 'male'),
 ('Leodora', 'female'),
 ('Webster', 'male'),
 ('Ashlen', 'female'),
 ('Talbert', 'male'),
 ('Reeva', 'female'),
 ('Linzy', 'female'),
 ('Veronique', 'female'),
 ('Fredrika', 'female'),
 ('Earle', 'male')]

In [ ]:
fset = [(atributos(n), g) for (n, g) in tagset]
train, test = fset[500:], fset[:500]

**Modelo de clasificación Naive Bayes**

In [ ]:
# entrenamiento del modelo NaiveBayes
classifier = nltk.NaiveBayesClassifier.train(train)

 **Verificación de algunas predicciones**

In [ ]:
classifier.classify(atributos('amanda'))

'female'

In [ ]:
classifier.classify(atributos('peter'))

'male'

**Performance del modelo**

In [ ]:
print(nltk.classify.accuracy(classifier, test))

0.734


In [ ]:
print(nltk.classify.accuracy(classifier, train))

0.7647770016120365


**Mejores atributos**

In [ ]:
def mas_atributos(nombre):
    atrib = {}
    atrib["primera_letra"] = nombre[0].lower()
    atrib["ultima_letra"] = nombre[-1].lower()
    for letra in 'abcdefghijklmnopqrstuvwxyz':
        atrib["count({})".format(letra)] = nombre.lower().count(letra)
        atrib["has({})".format(letra)] = (letra in nombre.lower())
    return atrib

In [ ]:
mas_atributos('jhon')

In [ ]:
fset = [(mas_atributos(n), g) for (n, g) in tagset]
train, test = fset[500:], fset[:500]
classifier2 = nltk.NaiveBayesClassifier.train(train)

In [ ]:
print(nltk.classify.accuracy(classifier2, test))

0.77


### Ejercicio de práctica

**Objetivo:** Construye un classificador de nombres en español usando el siguiente dataset: 
https://github.com/jvalhondo/spanish-names-surnames

1. **Preparación de los datos**: con un `git clone` puedes traer el dataset indicado a tu directorio en Colab, luego asegurate de darle el formato adecuado a los datos y sus features para que tenga la misma estructura del ejemplo anterior con el dataset `names` de nombres en ingles. 

* **Piensa y analiza**: ¿los features en ingles aplican de la misma manera para los nombres en español?

In [ ]:
# escribe tu código aquí


2. **Entrenamiento y performance del modelo**: usando el classificador de Naive Bayes de NLTK entrena un modelo sencillo usando el mismo feature de la última letra del nombre, prueba algunas predicciones y calcula el performance del modelo. 

In [ ]:
# escribe tu código aquí


3. **Mejores atributos:** Define una función como `atributos2()` donde puedas extraer mejores atributos con los cuales entrenar una mejor version del clasificador. Haz un segundo entrenamiento y verifica como mejora el performance de tu modelo. ¿Se te ocurren mejores maneras de definir atributos para esta tarea particular?

In [ ]:
# escribe tu código aquí


# Clasificación de documentos (email spam o no spam)

In [ ]:
!git clone https://github.com/pachocamacho1990/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df = pd.read_csv('datasets/email/csv/spam-apache.csv', names = ['clase','contenido'])
df['tokens'] = df['contenido'].apply(lambda x: word_tokenize(x))
df.head()

,clase,contenido,tokens
0,-1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...","[<, !, DOCTYPE, HTML, PUBLIC, ``, -//W3C//DTD,..."
1,1,> Russell Turpin:\n> > That depends on how the...,"[>, Russell, Turpin, :, >, >, That, depends, o..."
2,-1,Help wanted. We are a 14 year old fortune 500...,"[Help, wanted, ., We, are, a, 14, year, old, f..."
3,-1,Request A Free No Obligation Consultation!\nAc...,"[Request, A, Free, No, Obligation, Consultatio..."
4,1,Is there a way to look for a particular file o...,"[Is, there, a, way, to, look, for, a, particul..."


In [ ]:
df['tokens'].values[0]

In [ ]:
all_words = nltk.FreqDist([w for tokenlist in df['tokens'].values for w in tokenlist])
top_words = all_words.most_common(200)

def document_features(document):
    document_words = set(document)
    features = {}
    for word in top_words:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
document_features(df['tokens'].values[0])

In [ ]:
fset = [(document_features(texto), clase) for texto, clase in zip(df['tokens'].values, df['clase'].values)]
random.shuffle(fset)
train, test = fset[:200], fset[200:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [ ]:
print(nltk.classify.accuracy(classifier, test))

0.48


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
    contains(('us', 53)) = False               1 : -1     =      1.0 : 1.0
 contains(('could', 66)) = False               1 : -1     =      1.0 : 1.0
 contains(('money', 72)) = False               1 : -1     =      1.0 : 1.0
  contains(('days', 40)) = False               1 : -1     =      1.0 : 1.0
  contains(('need', 43)) = False               1 : -1     =      1.0 : 1.0


In [ ]:
df[df['clase']==-1]['contenido']

0      <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Tr...
2      Help wanted.  We are a 14 year old fortune 500...
3      Request A Free No Obligation Consultation!\nAc...
10     >\n>“µ×è¹µÑÇ ¡ÑºâÅ¡¸ØÃ¡Ô¨º¹ÍÔ¹àµÍÃìà¹çµ” \n>àµ...
11     ==============================================...
                             ...                        
243    ##############################################...
244    Wanna see sexually curious teens playing with ...
246    REQUEST FOR URGENT BUSINESS ASSISTANCE\n------...
248    Email marketing works!  There's no way around ...
249    Email marketing works!  There's no way around ...
Name: contenido, Length: 125, dtype: object

## Ejercicio de práctica


¿Como podrías construir un mejor clasificador de documentos?

0. **Dataset más grande:** El conjunto de datos que usamos fue muy pequeño, considera usar los archivos corpus que estan ubicados en la ruta: `datasets/email/plaintext/` 

1. **Limpieza:** como te diste cuenta no hicimos ningun tipo de limpieza de texto en los correos electrónicos. Considera usar expresiones regulares, filtros por categorias gramaticales, etc ... . 

---

Con base en eso construye un dataset más grande y con un tokenizado más pulido. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# escribe tu código aquí:


2. **Validación del modelo anterior:**  
---

una vez tengas el nuevo conjunto de datos más pulido y de mayor tamaño, considera el mismo entrenamiento con el mismo tipo de atributos del ejemplo anterior, ¿mejora el accuracy del modelo resultante?

In [ ]:
# escribe tu código aquí:


3. **Construye mejores atributos**: A veces no solo se trata de las palabras más frecuentes sino de el contexto, y capturar contexto no es posible solo viendo los tokens de forma individual, ¿que tal si consideramos bi-gramas, tri-gramas ...?, ¿las secuencias de palabras podrián funcionar como mejores atributos para el modelo?. Para ver si es así,  podemos extraer n-gramas de nuestro corpus y obtener sus frecuencias de aparición con `FreqDist()`, desarrolla tu propia manera de hacerlo y entrena un modelo con esos nuevos atributos, no olvides compartir tus resultados en la sección de comentarios. 

In [ ]:
# escribe tu código aquí:
